In [1]:
import json
import rdflib
import pandas as pd
import numpy as np
from uuid import uuid4

In [2]:
from rdflib.namespace import Namespace
from rdflib import URIRef, Literal, BNode

In [3]:
from pprint import pprint

In [4]:
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
from rdflib.namespace import RDFS, RDF

In [6]:
RDF

rdf.namespace.ClosedNamespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#')

In [7]:
RDFS

rdf.namespace.ClosedNamespace('http://www.w3.org/2000/01/rdf-schema#')

In [8]:
%%time
g = rdflib.Graph()
g.parse('input/migrationKB_16052021_1726.owl', format='turtle')

CPU times: user 41.4 ms, sys: 7.5 ms, total: 48.9 ms
Wall time: 47.5 ms


<Graph identifier=N0ace9eed85464c8d893a26bc1e4da6a7 (<class 'rdflib.graph.Graph'>)>

In [9]:
# define namespace
sioc = Namespace('http://rdfs.org/sioc/ns#')
sioc_t = Namespace('http://rdfs.org/sioc/types#')
rdfs = Namespace('http://www.w3.org/2000/01/rdf-schema#')
nee = Namespace('http://www.ics.forth.gr/isl/oae/core#')
schema = Namespace('http://schema.org/')
onyx = Namespace('http://www.gsi.dit.upm.es/ontologies/onyx/ns#')
wna = Namespace('http://www.gsi.dit.upm.es/ontologies/wnaffect/ns#')
dc = Namespace('http://purl.org/dc/elements/1.1/')
fibo_ind_ei_ei= Namespace('https://spec.edmcouncil.org/fibo/ontology/IND/EconomicIndicators/EconomicIndicators/')
fibo_fnd_dt_fd=Namespace('https://spec.edmcouncil.org/fibo/ontology/FND/DatesAndTimes/FinancialDates/')
fibo_fnd_rel_rel = Namespace('https://spec.edmcouncil.org/fibo/ontology/FND/Relations/Relations/')
fibo_fnd_utl_alx =Namespace('https://spec.edmcouncil.org/fibo/ontology/FND/Utilities/Analytics/')
fibo_fnd_arr_doc = Namespace('https://spec.edmcouncil.org/fibo/ontology/FND/Arrangements/Documents/')
rr
## binding
g.bind("sioc", sioc)
g.bind("sioc_t", sioc_t)
g.bind("rdfs", rdfs)
g.bind("wna", wna)
g.bind("nee", nee)
g.bind("dc", dc)
g.bind("schema", schema)
g.bind("onyx",onyx)
g.bind("fibo-ind-ei-ei", fibo_ind_ei_ei)
g.bind("fibo-fnd-dt-fd", fibo_fnd_dt_fd)
g.bind("fibo-fnd-rel-rel", fibo_fnd_rel_rel)
g.bind("fibo-fnd-utl-alx", fibo_fnd_utl_alx)
g.bind("fibo-fnd-arr-doc", fibo_fnd_arr_doc)

### populate
neutral_emotion = URIRef('http://www.gsi.dit.upm.es/ontologies/wnaffect/ns#neutral-emotion')
negative_emotion = URIRef('http://www.gsi.dit.upm.es/ontologies/wnaffect/ns#negative-emotion')
positive_emotion = URIRef('http://www.gsi.dit.upm.es/ontologies/wnaffect/ns#positive-emotion')

In [10]:
mgkb = "urn:absolute:mgKB/"

In [18]:
mgkb+'437312059282554880'

'urn:absolute:mgKB/437312059282554880'

In [91]:
len(g)

148

In [9]:
for smt in g:
    pprint(smt)

(rdflib.term.URIRef('http://www.ics.forth.gr/isl/oae/core#detectedAs'),
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#domain'),
 rdflib.term.URIRef('http://www.ics.forth.gr/isl/oae/core#Entity'))
(rdflib.term.URIRef('http://schema.org/Action'),
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#comment'),
 rdflib.term.Literal('[An action performed by a direct agent and indirect participants upon a direct object. Optionally happens at a location with the help of an inanimate instrument. The execution of the action may produce a result. Specific action sub-type documentation specifies the exact expectation of each argument/role.]\n\nAction that includes LikeAction, ShareAction, ReplyAction and QuoteAction.'))
(rdflib.term.URIRef('http://schema.org/interactionType'),
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#range'),
 rdflib.term.URIRef('http://schema.org/Action'))
(rdflib.term.URIRef('http://schema.org/latitude'),
 rdflib.term.URIRef('http://www.w3.org/

In [10]:
# df = pd.read_csv('input/tweets_migrationsKB_15052021.csv', index_col=0, low_memory=False)

### Generate random uuid for author_id and post_id

In [23]:
df.id = df.id.astype('str')

In [61]:
df['id_gen']= df['id'].parallel_apply(lambda x: str(uuid4()))

In [62]:
len(df) ==len(list(set(df.id_gen.tolist()))) ### all unique ids.

True

In [36]:
df.author_id = df.author_id.astype('str')

In [40]:
len(list(set(df.author_id.tolist())))

64602

In [42]:
author_ids = list(set(df.author_id.tolist()))

In [44]:
author_df = pd.DataFrame({'author_id': author_ids})

In [50]:
author_df['author_id_gen']= author_df['author_id'].parallel_apply(lambda x: str(uuid4()))

In [53]:
len(author_df) == len(list(set(author_df.author_id_gen.tolist())))

True

In [63]:
joined_df = pd.merge(df, author_df, how='inner', on=['author_id'])

In [64]:
len(list(set(joined_df.author_id_gen.tolist())))

64602

### get the primitive topic of the post

In [68]:
from ast import literal_eval

In [71]:
joined_df['prim_topic'] = joined_df['topics'].parallel_apply(lambda x: literal_eval(x)[0])

In [72]:
joined_df.prim_topic

0         12
1         48
2         48
3         16
4         27
          ..
177090    34
177091    48
177092     8
177093     4
177094    48
Name: prim_topic, Length: 177095, dtype: int64

In [109]:
joined_df.to_csv('input/migrationsKB_uuid.csv', index=False)

In [128]:
with open('input/entities_dict_extracted.json') as file:
    entities_dict = json.load(file)

In [130]:
entities_dict['1288506']

{'description': ' Syria (), officially the Syrian Arab Republic (), is a country in Western Asia, bordering Lebanon to the southwest, the Mediterranean Sea to the west, Turkey to the north, Iraq to the east, Jordan to the south, and Israel to the southwest. A country of fert',
 'entity': 'Syria',
 'url': 'https://en.wikipedia.org/wiki?curid=7515849'}

 ## test on 5 post 

In [11]:
df= pd.read_csv('input/migrationsKB_uuid.csv')

/home/yiyi/anaconda3/envs/migr/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (29,30,31,32,33,34,35,36,37,38,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [26]:
df.head()

,tokens,topics,id,texts,hashtags,entities,urls,processed_tokens,processed_text,processed_tm,...,twi_long,twi_lat,user_mentions,retweet_count,reply_count,like_count,quote_count,id_gen,author_id_gen,prim_topic
0,"['missed', 'beginning', 'aching', 'notice', 'y...","[12, 48, 32]",285908615561363456,"Completely missed the New Year beginning, too ...",['cardsagainsthumanity'],[],[],"['completely', 'missed', 'new', 'year', 'begin...",completely missed new year beginning engrossed...,"[18416, 374, 16452, 4179, 24601, 15425, 17097,...",...,-1.258131,53.100616,[],0.0,0.0,0.0,0.0,3165a1219c1642959c0f220a5826dc2a,294d47783c5e4408ab129a6a93fb0a15,12
1,"['trouble', 'call', 'cardsagainsthumanity']","[48, 17, 3]",313053990902456320,"If you're in trouble, you need to call... #car...",['cardsagainsthumanity'],[],['https://twitter.com/OperaticBacon/status/313...,"['trouble', 'need', 'cardsagainsthumanity']",trouble need cardsagainsthumanity,"[8381, 21418, 24601]",...,-1.258288,53.100615,[],0.0,0.0,0.0,0.0,a256d6d6c7ba48acbee98e1d4625802d,294d47783c5e4408ab129a6a93fb0a15,48
2,"['awesome', 'yourewelcome']","[48, 31, 1]",285915248383361026,Damien_Sandow is awesome #YoureWelcome,['YoureWelcome'],['Damien_Sandow'],[],"['damien_sandow', 'awesome', 'yourewelcome']",damien_sandow awesome yourewelcome,"[3849, 14438]",...,-0.637335,51.917134,[],0.0,0.0,0.0,0.0,5773523185ca42ad8b13100511b044ff,dfe2c45cee5c47d9a438191428ba9370,48
3,"['needed', 'asylum', 'ryan']","[16, 17, 48]",285921849446117376,Ryan was looking at me like I needed to be in ...,[],['Ryan'],[],"['ryan', 'looking', 'like', 'needed', 'asylum'...",ryan looking like needed asylum troublemaker came,"[21183, 3768, 11479]",...,-0.606182,52.285503,[],0.0,0.0,0.0,0.0,bae9a2b6b0ea49aab0d4df653150e92e,6ab49c01aef549b687d1780b877fc19e,16
4,"['stuck', 'illegal', 'work', 'lol', 'partying']","[27, 40, 45]",285997288793460736,@CapitalOfficial wish I was partying.... I'm s...,['illegal'],[],[],"['wish', 'partying', 'stuck', 'work', 'illegal...",wish partying stuck work illegal lol,"[24668, 4434, 21729, 19124, 21887]",...,-0.619785,53.570898,['CapitalOfficial'],1.0,1.0,1.0,0.0,56606f959bb54df5a1330b3520907119,24e422c1212543b784dc7615a4c476a8,27


In [ ]:
df[]

In [21]:
df['author_id_gen']= df['author_id_gen'].parallel_apply(lambda x: x.replace('-',''))

In [25]:
df['id_gen']= df['id_gen'].parallel_apply(lambda x: x.replace('-',''))

In [23]:
len(list(set(df.author_id_gen.tolist())))

64602

In [24]:
len(list(set(df.author_id.tolist())))

64602

In [27]:
df.to_csv('input/migrationsKB_uuid.csv', index=False)

In [85]:
test_df = df.sample(n=5)

In [93]:
df[~df['twi_lat'].isnull()].sample(n=5)

,tokens,topics,id,texts,hashtags,entities,urls,processed_tokens,processed_text,processed_tm,...,twi_long,twi_lat,user_mentions,retweet_count,reply_count,like_count,quote_count,id_gen,author_id_gen,prim_topic
3810,"['gay', 'mutilated', 'syrian', 'egaylity', 'is...","[31, 49, 46]",761442161468375041,"#Egaylity - Headless, mutilated body of gay S...",['Egaylity'],['Istanbul'],['http://dlvr.it/LyXxzV'],"['egaylity', 'headless', 'mutilated', 'body', ...",egaylity headless mutilated body gay syrian re...,"[6762, 20611, 4622, 19518, 3090, 5981, 21125, ...",...,0.020313,51.619671,[],0.0,0.0,0.0,0.0,41caf26480a94ce3bb5cfba1453af709,391ed5c44a1c4d5bb5b64b6725b061da,31
22256,"['policemen', 'news', 'graves', 'shallow', 'th...","[38, 13, 43]",594589824713039874,DE-News : Thai policemen measure shallow grave...,[],"['Songkhla', 'Thailand']",['http://dlvr.it/9dzY02'],"['denews', 'thai', 'policeman', 'measure', 'sh...",denews thai policeman measure shallow graf son...,"[23809, 5362, 9345, 4370, 18814, 6209, 5836, 1...",...,13.467578,52.591320,[],0.0,0.0,0.0,0.0,f7b634e3832e4533a3539bb821b21d01,37000cfe17c5461d841198901ae066b2,38
17833,"['demands', 'means', 'heard', 'madness', 'vote...","[12, 25, 23]",372478328911167488,@SkyNews I have never heard such nonsense this...,[],[],[],"['heard', 'nonsense', 'mean', 'migrant', 'use'...",heard nonsense mean migrant use vote demand of...,"[5746, 18424, 5344, 22425, 15585, 20373, 412, ...",...,-1.069142,50.814039,['SkyNews'],0.0,1.0,0.0,0.0,b98ac6b4830949acb6049172ab33011b,280f2f871de047a89c3a8258170831e0,12
94002,"['faces', 'spring', 'violets', 'home', 'welcom...","[26, 48, 40]",850420141015932930,Hellooooo!!! ❤ these little faces that #welcom...,"['welcome', 'home', 'welcomehome', 'violets', ...",[],['https://www.instagram.com/p/BSmEKy0D8Lr/'],"['hellooooo', 'little', 'face', 'welcome', 'ho...",hellooooo little face welcome home welcomehome...,"[11475, 17470, 23317, 16177, 13879]",...,4.273060,52.108060,[],0.0,0.0,0.0,0.0,6b8100005fda484abbfc196a5740e84e,8b1f12dc487b478f9ef24828afe7b51f,26
94864,"['drink', 'food', 'cooking', 'europe', 'family...","[19, 31, 38]",976506749967626242,I tried. #green #crafts #syria #cooking #fitne...,"['green', 'crafts', 'syria', 'cooking', 'fitne...",[],['https://www.instagram.com/p/Bgl9tgJA9UJ/'],"['tried', 'green', 'crafts', 'syria', 'cooking...",tried green crafts syria cooking fitness wine ...,"[16103, 13240, 15016, 1619, 17349, 18001, 1287...",...,0.750000,51.250000,[],0.0,0.0,0.0,0.0,33be2b4a09734333a4aaee9858bb2fe6,a279848f2d234bc2951d88f5ba144227,19


In [99]:
df.loc[17833]

tokens               ['demands', 'means', 'heard', 'madness', 'vote...
topics                                                    [12, 25, 23]
id                                                  372478328911167488
texts                @SkyNews I have never heard such nonsense this...
hashtags                                                            []
entities                                                            []
urls                                                                []
processed_tokens     ['heard', 'nonsense', 'mean', 'migrant', 'use'...
processed_text       heard nonsense mean migrant use vote demand of...
processed_tm         [5746, 18424, 5344, 22425, 15585, 20373, 412, ...
text_processed       never heard nonsense mean migrant use vote dem...
pred_sentiment                                                       0
hatespeech_pred                                                      0
hatespeech_label                                                normal
sentim

In [15]:
row_one = df.loc[25695]

In [28]:
row_one

tokens               ['losing', 'children', 'childrenofsyria', 'dan...
topics                                                    [49, 38, 25]
id                                                  437312059282554880
texts                "#Syria is in danger of losing a generation of...
hashtags                                  ['Syria', 'childrenofsyria']
entities                                                            []
urls                                         ['http://bit.ly/1nWqI5D']
processed_tokens     ['syria', 'danger', 'losing', 'generation', 'c...
processed_text       syria danger losing generation child childreno...
processed_tm                 [15016, 12155, 9430, 20018, 11101, 11709]
text_processed       syria danger losing generation child childreno...
pred_sentiment                                                       0
hatespeech_pred                                                      0
hatespeech_label                                                normal
sentim

In [ ]:
row_one 

In [33]:
row_one_cols =df.loc[25695,  df.columns.str.startswith('entity_')]

In [34]:
row_one_cols

entity_0     {'mention': 'syria', 'id': 1288506, 'start_pos...
entity_1                                                   NaN
entity_2                                                   NaN
entity_3                                                   NaN
entity_4                                                   NaN
entity_5                                                   NaN
entity_6                                                   NaN
entity_7                                                   NaN
entity_8                                                   NaN
entity_9                                                   NaN
entity_10                                                  NaN
entity_11                                                  NaN
entity_12                                                  NaN
entity_13                                                  NaN
entity_14                                                  NaN
entity_15                                              

In [40]:
row_one_dict = row_one.to_dict()

In [84]:
row_one_dict['Year']

2014

In [47]:
from ast import literal_eval

In [76]:
entities = [literal_eval(v) for e,v in row_one_dict.items() if e.startswith('entity_') if str(v)!='nan']

In [77]:
entities

[{'mention': 'syria', 'id': 1288506, 'start_pos': 1}]

In [75]:
str(entities[2])

'nan'

In [78]:
with open('input/entities_dict_extracted.json') as file:
    entities_dict=json.load(file)

In [80]:
entities_dict['1288506']

{'description': ' Syria (), officially the Syrian Arab Republic (), is a country in Western Asia, bordering Lebanon to the southwest, the Mediterranean Sea to the west, Turkey to the north, Iraq to the east, Jordan to the south, and Israel to the southwest. A country of fert',
 'entity': 'Syria',
 'url': 'https://en.wikipedia.org/wiki?curid=7515849'}

In [20]:
row_one['author_id_gen'].replace('-','')

'48131f007b72455d9b9189243a91cd74'

In [121]:
idx = row_one['id']

In [122]:
idx

'437312059282554880'

In [123]:
idx = URIRef(mgkb+idx)

In [124]:
idx

rdflib.term.URIRef('urn:absolute:mgKB/437312059282554880')

In [99]:
def add_triples_for_one_tweet(g, row):
    idx = row['id']
    
    # add sioc:id
    g.add((sioc.Post, sioc.id, Literal(id_gen)))
    # add dc:created
    created_at = row['created_at']
    g.add((sioc.Post, dc.created, Literal(created_at)))
    # add UserAccount and its id
    author_id_gen = row['author_id_gen']
    g.add((sioc.UserAccount, sioc.id, Literal(author_id_gen)))
    # add place (name, long, lat, country)
    place_name = row['place_fullname']
    lon = row['twi_long']
    lat = row['twi_lat']
    country_code = row['country_code']
    g.add((schema.Place, sioc.name, Literal(place_name)))
    g.add((schema.Place, schema.addressCountry, Literal(country_code)))
    g.add((schema.Place, schema.latitude, Literal(lat)))
    g.add((schema.Place, schema.longitude, Literal(lon)))

In [100]:
add_triples_for_one_tweet(g, row_one)

86da4fb1-7b49-4b64-bbc0-3bcdd8fbc4df


In [105]:
from datetime import datetime


now = datetime.now()
date_time = now.strftime("%m%d%Y_%H%M%S")

In [136]:
g.serialize(f"output/migrationsKB_{date_time}.t", format="turtle")

In [ ]:
g.add((sioc.Post, ))